<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/arm_angle_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# font

In [ ]:
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '24px'
      break
    }
  }
  '''))
get_ipython().events.register('pre_run_cell', increase_font)

# arm angle predict

In [ ]:
!pip install pybaseball
!pip install catboost

<IPython.core.display.Javascript object>

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 24.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pybaseball as bb
import catboost as cb
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import mean_squared_error, log_loss
import random

project_dir = 'drive/My Drive/arm-angle-predict'

<IPython.core.display.Javascript object>

In [ ]:
year = 2024
if os.path.isfile(f"{project_dir}/{year}-data.csv"):
  df = pd.read_csv(f"{project_dir}/{year}-data.csv")
else:
  df = bb.statcast(start_dt=f"{year}-03-01", end_dt=f"{year}-12-01")
  df['attack_zone'] = 'waste'
  df.loc[(df['plate_x'].between(-0.558, 0.558)) & (df['plate_z'].between(1.833,3.166)), 'attack_zone'] = 'heart'
  df.loc[(df['plate_x'].between(-1.108, 1.108)) & (df['plate_z'].between(1.166,3.833)) & (~df['attack_zone'].isin(['heart'])), 'attack_zone'] = 'shadow'
  df.loc[(df['plate_x'].between(-1.666, 1.666)) & (df['plate_z'].between(0.5,4.5)) & (~df['attack_zone'].isin(['heart', 'shadow'])), 'attack_zone'] = 'chase'
  df['swing'] = 0
  df.loc[df['description'].isin(['foul', 'swinging_strike','hit_into_play', 'foul_tip','swinging_strike_blocked']), 'swing'] = 1

  df = df.rename(columns={'player_name':'pitcher_name'})
  batter_bio = bb.playerid_reverse_lookup(df['batter'].unique().tolist())
  batter_bio['batter_name'] = batter_bio['name_last'].add(', ').add(batter_bio['name_first'])
  df = df.merge(batter_bio.loc[:, ['key_mlbam','batter_name']].rename(columns={'key_mlbam':'batter'}), how='left')
  df = df.rename(columns={'player_name':'pitcher_name'})
  df.to_csv(f"{project_dir}/{year}-data.csv", index=False)

<IPython.core.display.Javascript object>

In [ ]:
df['description'].unique()

<IPython.core.display.Javascript object>

array(['swinging_strike_blocked', 'swinging_strike', 'ball', 'foul',
       'called_strike', 'hit_into_play', 'blocked_ball', 'foul_tip',
       'foul_bunt', 'hit_by_pitch', 'missed_bunt', 'bunt_foul_tip',
       'pitchout'], dtype=object)

In [ ]:
df.dropna(subset=['plate_x', 'plate_z']).shape

<IPython.core.display.Javascript object>

(742825, 118)

In [ ]:
df['events'].unique()

<IPython.core.display.Javascript object>

array(['strikeout', nan, 'field_out', 'walk', 'single', 'double',
       'sac_fly', 'catcher_interf', 'force_out', 'hit_by_pitch',
       'fielders_choice', 'field_error', 'home_run',
       'grounded_into_double_play', 'double_play',
       'strikeout_double_play', 'fielders_choice_out', 'truncated_pa',
       'sac_bunt', 'triple', 'triple_play', 'sac_fly_double_play'],
      dtype=object)

In [ ]:
df.columns.values

<IPython.core.display.Javascript object>

array(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'pitcher_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year',
       'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0',
       'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle',
       'effective_speed', 'release_spin_rate', 'release_extension',
       'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5',
       'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
       'release_pos_y', 'estimated_ba_using_speedangle',
 

In [ ]:
strikes = [
    'swinging_strike_blocked', 'swinging_strike', 'foul',
    'called_strike', 'foul_tip', 'foul_bunt',
    'missed_bunt', 'bunt_foul_tip'
]
df['is_strike'] = 0
df.loc[df['description'].isin(strikes), 'is_strike'] = 1

df['k'] = 0
df.loc[df['events'].isin(['strikeout', 'strikeout_double_play']), 'k'] = 1
for col in ['on_1b', 'on_2b', 'on_3b']:
  df[col] = df[col].fillna(0).astype(bool)

comp_cols = [
    'pitch_type','pitch_name','on_3b','on_2b','on_1b',
    'inning','inning_topbot','outs_when_up','balls',
    'strikes','n_thruorder_pitcher','stand','p_throws',
    'sz_top','sz_bot','pfx_x','pfx_z','arm_angle',
    'release_speed','release_pos_x','release_extension',
    'release_pos_z','release_spin_rate','spin_axis',
    'bat_speed','swing_length','is_strike','k'
]
main_data = df.loc[:, comp_cols].copy()
main_data['spin_axis_adj'] = main_data['spin_axis'].sub(180)
main_data.loc[main_data['p_throws']=='L','spin_axis_adj'] = main_data.loc[main_data['p_throws']=='L','spin_axis_adj'].mul(-1)
main_data['spin_axis_adj'] = main_data['spin_axis_adj'].add(180)

<IPython.core.display.Javascript object>

In [ ]:
_df = main_data.copy()
for col in ['bat_speed','swing_length']:
  _df[col] = _df[col].fillna(-1)

<IPython.core.display.Javascript object>

In [ ]:
feats = [
    'on_3b','on_2b','on_1b',
    'inning','outs_when_up','balls',
    'strikes','n_thruorder_pitcher',
    'sz_top','sz_bot','pfx_x','pfx_z','arm_angle',
    'release_speed','release_pos_x','release_extension',
    'release_pos_z','release_spin_rate','spin_axis',
    'bat_speed','swing_length'
]
target = 'is_strike'
model_data = _df.dropna(subset=feats+[target])

folds = 3
kf = KFold(folds, shuffle=True)
s = pd.Series()
for train_idx, test_idx in kf.split(model_data):
  train = model_data.iloc[train_idx]
  test = model_data.iloc[test_idx]
  model = cb.CatBoostClassifier(eta=0.001, verbose=False)
  model.fit(train.loc[:, feats], train[target])
  _s = pd.DataFrame(model.predict_proba(test.loc[:, feats]), index=test.index)
  _s = _s.loc[:, [1]]
  s = pd.concat([s, _s])

model_data[f'{target}_cb'] = s[1]

feats = [
    'on_3b','on_2b','on_1b',
    'inning','outs_when_up','balls',
    'strikes','n_thruorder_pitcher',
    'sz_top','sz_bot','pfx_x','pfx_z','arm_angle',
    'release_speed','release_pos_x','release_extension',
    'release_pos_z','release_spin_rate','spin_axis',
    'bat_speed','swing_length', 'is_strike_cb'
]
target = 'k'
_model_data = model_data.loc[model_data['strikes']==2]
folds = 3
kf = KFold(folds, shuffle=True)
s = pd.Series()
for train_idx, test_idx in kf.split(_model_data):
  train = _model_data.iloc[train_idx]
  test = _model_data.iloc[test_idx]
  model = cb.CatBoostClassifier(eta=0.001, verbose=False)
  model.fit(train.loc[:, feats], train[target])
  _s = pd.DataFrame(model.predict_proba(test.loc[:, feats]), index=test.index)
  _s = _s.loc[:, [1]]
  s = pd.concat([s, _s])

model_data[f'{target}_cb'] = s[1]


<IPython.core.display.Javascript object>

<ipython-input-7-7a91cd7d05b2>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data[f'{target}_cb'] = s[1]
<ipython-input-7-7a91cd7d05b2>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data[f'{target}_cb'] = s[1]


In [ ]:
model_data.loc[model_data['strikes']==2,'k_round'] = 0
model_data.loc[(model_data['strikes']==2)&(model_data['is_strike_cb']>.5), 'k_round'] = 1

model_data.loc[model_data['strikes']==2,'k_fill'] = model_data.loc[(model_data['strikes']==2), 'is_strike_cb']

<IPython.core.display.Javascript object>

<ipython-input-8-22d32a580d0f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.loc[model_data['strikes']==2,'k_round'] = 0
<ipython-input-8-22d32a580d0f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.loc[model_data['strikes']==2,'k_fill'] = model_data.loc[(model_data['strikes']==2), 'is_strike_cb']


In [ ]:
target = 'is_strike'
log_loss(model_data[target], model_data[f'{target}_cb'])

<IPython.core.display.Javascript object>

0.6164667764131438

In [ ]:
target = 'k'
log_loss(model_data.loc[model_data['strikes']==2,target], model_data.loc[model_data['strikes']==2,f"{target}_cb"])

<IPython.core.display.Javascript object>

0.4713604787824396

In [ ]:
log_loss(model_data.loc[model_data['strikes']==2,'k'], model_data.loc[model_data['strikes']==2,f"{target}_round"])

<IPython.core.display.Javascript object>

17.283931295520564

In [ ]:
log_loss(model_data.loc[model_data['strikes']==2,'k'], model_data.loc[model_data['strikes']==2,f"{target}_fill"])

<IPython.core.display.Javascript object>

0.6294758922155188

In [ ]:
train_feats = [
    'index','pitch_type','pitch_name','on_3b','on_2b',
    'on_1b','inning','inning_topbot','outs_when_up',
    'balls','strikes','n_thruorder_pitcher','stand',
    'p_throws','sz_top','sz_bot','pfx_x','pfx_z',
    'arm_angle','release_speed','release_pos_x',
    'release_extension','release_pos_z',
    'release_spin_rate','spin_axis','bat_speed',
    'swing_length','is_strike','k'
]

test_feats = [
    'index','pitch_type','pitch_name','on_3b','on_2b',
    'on_1b','inning','inning_topbot','outs_when_up',
    'balls','strikes','n_thruorder_pitcher','stand',
    'p_throws','sz_top','sz_bot','pfx_x','pfx_z',
    'arm_angle','release_speed','release_pos_x',
    'release_extension','release_pos_z',
    'release_spin_rate','spin_axis','bat_speed',
    'swing_length'
]
solution_feats = [
    'index', 'k'
]

<IPython.core.display.Javascript object>

In [ ]:
main_data.shape

<IPython.core.display.Javascript object>

(757713, 29)

In [ ]:
main_data.columns.values

<IPython.core.display.Javascript object>

array(['pitch_type', 'pitch_name', 'on_3b', 'on_2b', 'on_1b', 'inning',
       'inning_topbot', 'outs_when_up', 'balls', 'strikes',
       'n_thruorder_pitcher', 'stand', 'p_throws', 'sz_top', 'sz_bot',
       'pfx_x', 'pfx_z', 'arm_angle', 'release_speed', 'release_pos_x',
       'release_extension', 'release_pos_z', 'release_spin_rate',
       'spin_axis', 'bat_speed', 'swing_length', 'is_strike', 'k',
       'spin_axis_adj'], dtype=object)

In [ ]:
subset = [
    'pitch_type','pitch_name','on_3b','on_2b',
    'on_1b','inning','inning_topbot','outs_when_up',
    'balls','strikes','n_thruorder_pitcher','stand',
    'p_throws','sz_top','sz_bot','pfx_x','pfx_z',
    'arm_angle','release_speed','release_pos_x',
    'release_extension','release_pos_z',
    'release_spin_rate','spin_axis','is_strike','k'
]
train = main_data.dropna(subset=subset).sample(frac=1, random_state=2025020446).reset_index(drop=True)
train = train.reset_index()
test = train.loc[train['strikes']==2].sample(50000, random_state=2025020451)
train = train.loc[~train.index.isin(test.index.values)]
solution = test.loc[:, ['index','k']]
sample_solution = solution.copy()
sample_solution['k'] = test['k'].mean()

train = train.loc[:, train_feats]
test = test.loc[:, test_feats]
solution = solution.loc[:, solution_feats]
solution['Usage'] = 'Private'
solution.loc[solution.index[range(0,500)], 'Usage'] = 'Public'
sample_solution = sample_solution.loc[:, solution_feats]

train.to_csv(f"{project_dir}/train.csv", index=False)
test.to_csv(f"{project_dir}/test.csv", index=False)
solution.to_csv(f"{project_dir}/solution.csv", index=False)
sample_solution.to_csv(f"{project_dir}/sample_solution.csv", index=False)

<IPython.core.display.Javascript object>